In [0]:
from fastai.core import *
from fastai.vision import *

## Download HASYv2 dataset

- https://arxiv.org/abs/1701.08380
- https://zenodo.org/record/259444#.XTcKKJMzbm0


In [0]:
# !wget --header="Host: zenodo.org" --header="User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9,ta;q=0.8" --header="Referer: https://zenodo.org/record/259444" --header="Cookie: session=436d12c44e21af6d_5d370a26.Sr1s-0tpOe6FVmOM5tGcX2Fuaak; __atuvc=1%7C30; __atuvs=5d370a2897888193000; _pk_ses.57.a333=*; _pk_id.57.a333=94576573ef886f63.1563888171.1.1563888172.1563888171." --header="Connection: keep-alive" "https://zenodo.org/record/259444/files/HASYv2.tar.bz2?download=1" -O "HASYv2.tar.bz2" -c

In [0]:
filename = 'HASYv2.tar.bz2'
!mkdir datav3

In [0]:
# !cd datav2; 

In [0]:
!tar -xjf datav2/{filename} -C datav3/

In [0]:
# pathv2 = Path("datav2")
pathv3 = Path("datav3")

In [153]:
pathv3.ls()

[PosixPath('datav3/hasy-data'),
 PosixPath('datav3/symbols.csv'),
 PosixPath('datav3/hasy_tools.py'),
 PosixPath('datav3/README.txt'),
 PosixPath('datav3/verification-task'),
 PosixPath('datav3/hasy-data-labels.csv'),
 PosixPath('datav3/classification-task')]

In [154]:
symbols_df = pd.read_csv(pathv3/'symbols.csv')
symbols_df.tail()

,symbol_id,latex,training_samples,test_samples
364,1385,\O,213,24
365,1394,\o,282,32
366,1395,\ae,257,29
367,1396,\AE,134,15
368,1400,\guillemotleft,96,11


In [0]:
pathv3img = pathv3/'hasy-data'

In [156]:
len(pathv3img.ls())

168233

In [157]:
lables_df = pd.read_csv(pathv3/'hasy-data-labels.csv')
lables_df.tail()

,path,symbol_id,latex,user_id
168228,hasy-data/v2-168228.png,1400,\guillemotleft,16925
168229,hasy-data/v2-168229.png,1400,\guillemotleft,16925
168230,hasy-data/v2-168230.png,1400,\guillemotleft,16925
168231,hasy-data/v2-168231.png,1400,\guillemotleft,16925
168232,hasy-data/v2-168232.png,1400,\guillemotleft,16925


In [158]:
!ls -lh ./datav3/hasy-data/v2-00000.png

-rw-r--r-- 1 1000 1000 243 Jan 24  2017 ./datav3/hasy-data/v2-00000.png


In [159]:
(pathv3/'hasy-data').ls()[:3]

[PosixPath('datav3/hasy-data/v2-119339.png'),
 PosixPath('datav3/hasy-data/v2-83012.png'),
 PosixPath('datav3/hasy-data/v2-47840.png')]

In [0]:
il = ImageList.from_csv(pathv3, 'hasy-data-labels.csv')
sp = il.split_by_rand_pct()
lf = sp.label_from_df()
# tf = lf.transform(tfms, size=32)
# db = tf.databunch()

In [0]:
# db.show_batch(rows=3)

In [162]:
il

ImageList (168233 items)
Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32),Image (3, 32, 32)
Path: datav3

In [163]:
for item in il.items:
  print(item)
  break

datav3/hasy-data/v2-00000.png


In [165]:
lf.train.x.items

array(['datav3/hasy-data/v2-00000.png', 'datav3/hasy-data/v2-00001.png', 'datav3/hasy-data/v2-00002.png',
       'datav3/hasy-data/v2-00003.png', ..., 'datav3/hasy-data/v2-168229.png', 'datav3/hasy-data/v2-168230.png',
       'datav3/hasy-data/v2-168231.png', 'datav3/hasy-data/v2-168232.png'], dtype='<U31')

In [0]:
# lf.train.y.classes

In [0]:
copy_fn = lambda src,dst: shutil.copy(src,dst)

In [0]:
dst_dir = 'datav3/hasy-data-imagenet-style'
# !rm -rf dst_dir

dst_dir_train = dst_dir+'/train'
dst_dir_valid = dst_dir+'/valid'

!mkdir {dst_dir}
!mkdir {dst_dir_train}
!mkdir {dst_dir_valid}

In [0]:
# Create label folders
# lf.train.y.items

# lf.train.y.c2i
i2c_train = {v:k for k,v in lf.train.y.c2i.items()}
i2c_valid = {v:k for k,v in lf.valid.y.c2i.items()}

In [0]:
# set(i2c_train.values()) == set(i2c_valid.values())

# Create all the base folders

def create_lables_folder(base_folder):
  for foldername in progress_bar(set(i2c_train.values())):
    full_folder = Path(f"{base_folder}/{foldername}")
    print(full_folder)
    full_folder.mkdir()
    
create_lables_folder(dst_dir_train)
create_lables_folder(dst_dir_valid)

In [0]:
# i2c_train

In [175]:
def copy_images_to_folder(base_folder, lf_tv):
  for fname,label_class_id in progress_bar(list(zip(lf_tv.x.items, lf_tv.y.items))):
    label_class = i2c_train[label_class_id]
    #print(fname, f"{base_folder}/{label_class}", label_class_id)
    copy_fn(fname, f"{base_folder}/{label_class}")
    #if i==10 : break

copy_images_to_folder(dst_dir_train, lf.train)
copy_images_to_folder(dst_dir_valid, lf.valid)

In [176]:
set(lf.valid.y.items) == set(lf.train.y.items) 

True

-rw-r--r-- 1 root 11086288 Jul 23 15:41 mathsymbol-best.h5
-rw-r--r-- 1 root  3668308 Jul 23 15:42 mathsymbol-best.tflite
-rw-r--r-- 1 root 11086520 Jul 23 14:23 mathsymbol.h5
-rw-r--r-- 1 root  3668332 Jul 23 14:23 mathsymbol.tflite
-rw-r--r-- 1 root 11086200 Jul 23 14:45 mathsymbol-v1.h5
-rw-r--r-- 1 root  3668320 Jul 23 14:46 mathsymbol-v1.tflite


## Load data using Keras - ImageDataGen

In [177]:
# Final ImageNet Style Path

# path_imagenet_style = './datav3/hasy-data-imagenet-style'
# path_imagenet_style

PosixPath('datav3/hasy-data-imagenet-style')

In [0]:
import math

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.datasets import mnist
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

In [0]:
# len(set(lf.train.y.items)), lf.c
# NUM_CLASSES = lf.c
NUM_CLASSES = 369

In [0]:
# lf.train.x

# DataLoaders

In [0]:
train_data_dir = './datav3/hasy-data-imagenet-style/train'
valid_data_dir = './datav3/hasy-data-imagenet-style/valid'
batch_size = 64
img_height, img_width = 32,32
nb_epochs = 5

In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=False)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)


In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

validation_generator = test_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

Found 134587 images belonging to 369 classes.
Found 33646 images belonging to 369 classes.


In [0]:
# Export labels
import json

with open('HASYv2-lables.json', 'w') as f:
  f.write(json.dumps(train_generator.class_indices))

# Model

In [204]:
NUM_CLASSES

369

In [27]:
## Model 
inputs = Input(shape=(32, 32, 3), name='input')

x = Conv2D(24, kernel_size=(6, 6), strides=1)(inputs)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)

x = Conv2D(48, kernel_size=(5, 5), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)


x = Conv2D(64, kernel_size=(4, 4), strides=2)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)

x = tf.keras.layers.Flatten()(x)
x = Dense(200)(x)
x = BatchNormalization(scale=False, beta_initializer=Constant(0.01))(x)
x = Activation('relu')(x)
x = Dropout(rate=0.25)(x)

predications = Dense(NUM_CLASSES, activation='softmax', name='output')(x)

model = Model(inputs=inputs, outputs=predications)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 27, 27, 24)        2616      
_________________________________________________________________
batch_normalization (BatchNo (None, 27, 27, 24)        72        
_________________________________________________________________
activation (Activation)      (None, 27, 27, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 48)        28848     
_________________________________________________________________
batch_normalization_1 (Batch (None, 12, 12, 48)        144       
_________________________________________________________________
activation_1 (Activation)    (None, 12, 12, 48)        0     

In [0]:
lr_decay = lambda epoch: 0.001 + 0.02 * math.pow(1.0 / math.e, epoch / 3.0)
decay_callback = LearningRateScheduler(lr_decay, verbose=1)

In [0]:
# model.fit_generator(
#         train_generator,
#         steps_per_epoch=2000,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=800)

In [0]:
model.load_weights('HASYv2-best.h5')

In [30]:
# history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, 
#                     validation_data=(x_test, y_test), callbacks=[decay_callback])


history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 10, callbacks=[decay_callback])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.021.
Epoch 1/10
2102/2102 [==============================] - 144s 68ms/step - loss: 0.6832 - acc: 0.7863 - val_loss: 0.8908 - val_acc: 0.7717

Epoch 00002: LearningRateScheduler reducing learning rate to 0.015330626211475785.
Epoch 2/10
2102/2102 [==============================] - 144s 68ms/step - loss: 0.5929 - acc: 0.8061 - val_loss: 0.7922 - val_acc: 0.7931

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01126834238065184.
Epoch 3/10
2102/2102 [==============================] - 143s 68ms/step - loss: 0.5217 - acc: 0.8228 - val_loss: 0.7981 - val_acc: 0.8026

Epoch 00004: LearningRateScheduler reducing learning rate to 0.008357588823428847.
Epoch 4/10
2102/2102 [==============================] - 144s 68ms/step - loss: 0.4689 - acc: 0.8367 - val_loss: 0.8324 - val_acc: 0.8053

Epoch 00005: LearningRateScheduler reducing learning rate to 0.006271942762314537.
Epoch 5/10
2102/2102 [==========================

In [24]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 4, callbacks=[decay_callback])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.021.
Epoch 1/4
2102/2102 [==============================] - 144s 68ms/step - loss: 0.7466 - acc: 0.7689 - val_loss: 0.8400 - val_acc: 0.7827

Epoch 00002: LearningRateScheduler reducing learning rate to 0.015330626211475785.
Epoch 2/4
2102/2102 [==============================] - 143s 68ms/step - loss: 0.6411 - acc: 0.7947 - val_loss: 0.7672 - val_acc: 0.7924

Epoch 00003: LearningRateScheduler reducing learning rate to 0.01126834238065184.
Epoch 3/4
2102/2102 [==============================] - 141s 67ms/step - loss: 0.5684 - acc: 0.8118 - val_loss: 0.7085 - val_acc: 0.8056

Epoch 00004: LearningRateScheduler reducing learning rate to 0.008357588823428847.
Epoch 4/4
2102/2102 [==============================] - 141s 67ms/step - loss: 0.5071 - acc: 0.8260 - val_loss: 0.7248 - val_acc: 0.8085


In [13]:
# Save & Export

model.save('HASYv2.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('HASYv2.h5')
tflite_model = converter.convert()
open('HASYv2.tflite', 'wb').write(tflite_model)

W0723 17:14:24.156207 140169430407040 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0723 17:14:24.157888 140169430407040 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0723 17:14:24.163671 140169430407040 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is depr

1902256

In [0]:
#Download

try:
    from google.colab import files
    files.download('HASYv2.tflite')
except:
    print("Skip downloading")

In [25]:
# Save & Export

model.save('HASYv2-best.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('HASYv2-best.h5')
tflite_model = converter.convert()
open('HASYv2-best.tflite', 'wb').write(tflite_model)

1902256

In [0]:
#Download

try:
    from google.colab import files
    files.download('HASYv2-best.tflite')
except:
    print("Skip downloading")

In [0]:
# Save & Export

model.save('HASYv2-best1.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('HASYv2-best1.h5')
tflite_model = converter.convert()
open('HASYv2-best1.tflite', 'wb').write(tflite_model)

#Download

# Export labels
import json

with open('HASYv2-lables.json', 'w') as f:
  f.write(json.dumps(train_generator.class_indices))
  

indices_to_class = { v:k for k,v in train_generator.class_indices.items()}

In [0]:
import pandas as pd

In [0]:

df1 = pd.DataFrame([item for item in indices_to_class.items()])
df1.columns = ['index', 'labels']
df1.to_csv('HASYv2-lables.csv', header=True, index=False)

try:
    from google.colab import files
    files.download('HASYv2-best1.tflite')
    files.download('HASYv2-lables.csv')
    files.download('HASYv2-lables.json')
except:
    print("Skip downloading")
    